# Practice GBM

In [113]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

In [114]:
df = pd.DataFrame({
    'height' : [1.6, 1.7, 1.6, 1.5, 1.5,1.4],
    'favorite color' : ['red', 'green','blue','red','red','red'],
    'sex' : [0,0,0,1,1,1],
    'weight' :[40,70,50,55,44,56]
})
print(df)

# 범주형 변수인 color열을 0,1으로 인코딩 해준다.
df= pd.get_dummies(df)
print(df)

   height favorite color  sex  weight
0     1.6            red    0      40
1     1.7          green    0      70
2     1.6           blue    0      50
3     1.5            red    1      55
4     1.5            red    1      44
5     1.4            red    1      56
   height  sex  ...  favorite color_green  favorite color_red
0     1.6    0  ...                     0                   1
1     1.7    0  ...                     1                   0
2     1.6    0  ...                     0                   0
3     1.5    1  ...                     0                   1
4     1.5    1  ...                     0                   1
5     1.4    1  ...                     0                   1

[6 rows x 6 columns]


## 몸무게를 예측해보자 
- 초기의 업데이트 값들은 잔차도 크고 예측값도 실제 몸무게와 많이 다름 그러나 뒤로 갈수록 잔차는 0에 가깝고 실제 몸무게 값과 비슷한 결과를 도출해냄

In [115]:
# Step1 : 모형 초기화
x= df.loc[:,df.columns!='weight']
y = df['weight']
F0 = np.mean(y)
print("F0 : {0:.4f}".format(F0));print('='*50)
# Step2 : 잔차 구하기
r1 = y - F0
print('residual(y - F0) : \n', r1);print('='*50)

# Step3 : Step2에서 구한 잔차에 깁이가 2인 DecisionTree를 fitting 시켜 잔차예측값을 구한다.
model = DecisionTreeRegressor(max_depth=2)
r1_fit = model.fit(x,r1)
gamma1 = r1_fit.predict(x)
print('잔차 예측값 : {}'.format(gamma1));print('='*50)

# Step4 : 예측값 업데이트
eta = 0.1 # learning rate
F1 = F0 + eta*gamma1
print('예측된 몸무게: {}'.format(F1));print('='*50)

F0 : 52.5000
residual(y - F0) : 
 0   -12.5
1    17.5
2    -2.5
3     2.5
4    -8.5
5     3.5
Name: weight, dtype: float64
잔차 예측값 : [-5.25 17.5  -5.25 -5.25 -5.25  3.5 ]
예측된 몸무게: [51.975 54.25  51.975 51.975 51.975 52.85 ]


In [116]:
F0 = np.mean(y) # 초기화 모형
tree_model = DecisionTreeRegressor(max_depth=2) #약한 학습기
eta=0.1 # 학습률

for t in range(100):
    F_old = F0
    r = y - F_old # 잔차
    r_fit = tree_model.fit(x,r) # 잔차 학습
    r_pred = r_fit.predict(x) # 잔차 예측
    F0 = F_old + eta*r_pred # 잔차 예측값
    if t >90:
        print('{}번째  잔차예측값 {} 몸무게 예측값{}'.format(t+1,np.round(r_pred,1),np.round(F0,1)))
    

92번째  잔차예측값 [-0.  0.  0.  0.  0.  0.] 몸무게 예측값[40.  70.  50.  49.5 49.5 56. ]
93번째  잔차예측값 [-0.  0.  0.  0.  0.  0.] 몸무게 예측값[40.  70.  50.  49.5 49.5 56. ]
94번째  잔차예측값 [-0.  0. -0.  0.  0.  0.] 몸무게 예측값[40.  70.  50.  49.5 49.5 56. ]
95번째  잔차예측값 [-0.  0.  0.  0.  0.  0.] 몸무게 예측값[40.  70.  50.  49.5 49.5 56. ]
96번째  잔차예측값 [-0.  0.  0. -0. -0.  0.] 몸무게 예측값[40.  70.  50.  49.5 49.5 56. ]
97번째  잔차예측값 [-0.  0.  0.  0.  0.  0.] 몸무게 예측값[40.  70.  50.  49.5 49.5 56. ]
98번째  잔차예측값 [-0.  0.  0.  0.  0.  0.] 몸무게 예측값[40.  70.  50.  49.5 49.5 56. ]
99번째  잔차예측값 [-0.  0.  0.  0.  0.  0.] 몸무게 예측값[40.  70.  50.  49.5 49.5 56. ]
100번째  잔차예측값 [-0.  0. -0.  0.  0.  0.] 몸무게 예측값[40.  70.  50.  49.5 49.5 56. ]


In [117]:
np.round(mean_squared_error(y,F0),2)

10.08

## sklearn GradientBoostingRegressor를 이용해보자
- GradientBoostRegressor로 적합 <br> 10-fold 교차검증을 통해 MSE를 계산
- GridSearchCV를 활용해 hyper parameter 추출


In [123]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GridSearchCV

# evaluate the model
model = GradientBoostingRegressor()
cv = RepeatedKFold(n_splits=3,n_repeats=3)
n_scores = cross_val_score(model, x,y,scoring='neg_mean_squared_error',cv=cv,error_score='raise')
print('MSE : {0:.4f}'.format(-np.mean(n_scores)))
print('='*50)

# GridSearCV를 활용해 hyperparameter 추출
params = {
    'n_estimators' :[100,200,300],
    'learning_rate' : [0.001,0.01]
}
grid_cv = GridSearchCV(model,param_grid=params,cv=2,verbose=1)
grid_cv.fit(x,y)
print('최적 하이퍼파라미터 :\n', grid_cv.best_params_)
print('최적 정확도 :\n', grid_cv.best_score_)
print('='*50)


MSE : 228.2082
Fitting 2 folds for each of 6 candidates, totalling 12 fits
최적 하이퍼파라미터 :
 {'learning_rate': 0.001, 'n_estimators': 100}
최적 정확도 :
 -0.013944920652831816
